In [ ]:
# mount on google drive
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import os
os.chdir('/content/drive/MyDrive/satellite-altimetry-course')


In [ ]:
# !pip install cartopy
# !pip install pyrsimg
# !apt-get install hdf5-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 31.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  hdf5-tools
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 347 kB of archives.
After this operation, 1,255 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 hdf5-tools amd64 1.10.7+repack-4ubuntu2 [347 kB]
Fetched 347 kB in 1s (646 kB/s)
Selecting previously unselected package hdf5-tools.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../hdf5-tools_1.10.7+repack-4ubuntu2_amd64.deb ...
Unpacking hdf5-tools (1.10.7+repack-4ubuntu2) ...
Setting up hdf5-tools (1.10.7+repack-4ubuntu2) ...
Processing triggers for man-db (2.10.2-1) ...


### **ICESat-2数据处理**

In [ ]:
import h5py
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from pyrsimg import readTiff, imgShow, second_to_dyr


In [ ]:
path_atl06 = 'data/laser/processed_ATL06_20200806013350_06370802_006_01.h5'
path_rsimg = 'data/rsimg/s2_gla_region_20220707.tif'


In [ ]:
!h5ls $path_atl06
# !h5ls $path_atl06/gt1l/land_ice_segments
# !h5ls $path_atl06/orbit_info


METADATA                 Group
ancillary_data           Group
gt1l                     Group
gt1r                     Group
gt2l                     Group
gt2r                     Group
gt3l                     Group
gt3r                     Group
orbit_info               Group
quality_assessment       Group


### Visualization

## 读入多波束数据并合并为一个group

In [ ]:
vars_atl06 = {}
beams = ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r']
with h5py.File(path_atl06,'r') as atl06:
    for beam in beams:
        ## gt1l
        vars_atl06['lat_'+beam] = atl06[beam+'/land_ice_segments/latitude'][:]
        vars_atl06['lon_'+beam] = atl06[beam+'/land_ice_segments/longitude'][:]
        vars_atl06['elev_'+beam] = atl06[beam+'/land_ice_segments/h_li'][:]
        vars_atl06['time_delta_'+beam] = atl06[beam+'/land_ice_segments/delta_time'][:] ## Seconds start with the reference sdp time(2018-01-01).
        ### quality
        vars_atl06['qual_'+beam] = atl06[beam+'/land_ice_segments/atl06_quality_summary'][:]
    ## orbit information.
    vars_atl06['cycle'] = atl06['orbit_info/cycle_number'][0]
    vars_atl06['track'] = atl06['orbit_info/rgt'][0]
    vars_atl06['orient'] = atl06['orbit_info/sc_orient'][0]

## 2. remote sensing image
rsimg = readTiff(path_rsimg)



In [ ]:
region = [94.56, 30.38, 95.04, 30.77]
processed_vars_atl06 = {}

for beam in beams:
    ## 排除异常点，及选取质量好的数据
    ids_beam = np.where((vars_atl06['lat_'+beam]>region[1]) \
                        & (vars_atl06['lat_'+beam]<region[3]) \
                        & (vars_atl06['lon_'+beam]>region[0]) \
                        & (vars_atl06['lon_'+beam]<region[2]) \
                        & (vars_atl06['elev_'+beam]<9000) \
                        & (vars_atl06['qual_'+beam]==0))
    processed_vars_atl06['lat_'+beam] = vars_atl06['lat_'+beam][ids_beam]
    processed_vars_atl06['lon_'+beam] = vars_atl06['lon_'+beam][ids_beam]
    processed_vars_atl06['elev_'+beam] = vars_atl06['elev_'+beam][ids_beam]
    processed_vars_atl06['time_delta_'+beam] = vars_atl06['time_delta_'+beam][ids_beam]

## orbit information
processed_vars_atl06['cycle'] = vars_atl06['cycle']
processed_vars_atl06['track'] = vars_atl06['track']
processed_vars_atl06['orient'] = vars_atl06['orient']


In [ ]:
### Apply time conversion (seconds to decimal year)
### Seconds (from 2018-01-01 00:00:00) to decimal year
print(processed_vars_atl06['time_delta_gt1l'][0:10])
for beam in beams:
    processed_vars_atl06['time_dyr_'+beam] = second_to_dyr(processed_vars_atl06['time_delta_gt1l'], time_start = '2018-01-01 00:00:00')
print(processed_vars_atl06['time_dyr_gt1l'][0:10])


[81912883.01548333 81912883.01832423 81912883.02116695 81912883.02400978
 81912883.0268495  81912883.02968517 81912883.03251266 81912883.0353311
 81912883.0381452  81912883.04095736]
[2020.5958093 2020.5958093 2020.5958093 2020.5958093 2020.5958093
 2020.5958093 2020.5958093 2020.5958093 2020.5958093 2020.5958093]


## write out to .h5 file.

In [ ]:
# file_out = 'data/laser/isat2_gla_region.h5'
# with h5py.File(file_out, "w") as f_out:
#     [f_out.create_dataset(key, data=processed_vars_atl06[key]) for key in processed_vars_atl06.keys()]
#     print('written file:', (file_out))

